# Part 2: Predictive Analysis - Review Score Prediction
## Predicting Customer Satisfaction Based on Order Features

This notebook builds a supervised machine learning model to predict whether a customer review will be high (4-5 stars) or low (1-3 stars) based on order characteristics.

**Business Objective**: Enable proactive customer satisfaction management by identifying orders likely to receive poor reviews before the review is submitted.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## Data Loading and Initial Exploration

In [ ]:
# Load datasets
orders = pd.read_csv('Data/olist_orders_dataset.csv')
order_items = pd.read_csv('Data/olist_order_items_dataset.csv')
customers = pd.read_csv('Data/olist_customers_dataset.csv')
products = pd.read_csv('Data/olist_products_dataset.csv')
payments = pd.read_csv('Data/olist_order_payments_dataset.csv')
reviews = pd.read_csv('Data/olist_order_reviews_dataset.csv')
sellers = pd.read_csv('Data/olist_sellers_dataset.csv')
category_translation = pd.read_csv('Data/product_category_name_translation.csv')

print(f"Reviews dataset shape: {reviews.shape}")
print(f"\nReview score distribution:")
print(reviews['review_score'].value_counts().sort_index())

# Calculate review score statistics
print(f"\nReview score statistics:")
print(f"Mean: {reviews['review_score'].mean():.2f}")
print(f"Median: {reviews['review_score'].median():.1f}")
print(f"Mode: {reviews['review_score'].mode().iloc[0]}")

## Feature Engineering and Data Preparation

In [ ]:
# Create target variable: high review (4-5) vs low review (1-3)
reviews['is_high_review'] = (reviews['review_score'] >= 4).astype(int)

print("Target variable distribution:")
target_dist = reviews['is_high_review'].value_counts()
print(f"Low reviews (1-3): {target_dist[0]} ({target_dist[0]/len(reviews)*100:.1f}%)")
print(f"High reviews (4-5): {target_dist[1]} ({target_dist[1]/len(reviews)*100:.1f}%)")

# Check for class imbalance
class_ratio = target_dist[1] / target_dist[0]
print(f"\nClass ratio (High:Low): {class_ratio:.2f}:1")
if class_ratio > 2 or class_ratio < 0.5:
    print("⚠️  Significant class imbalance detected - will need to address in modeling")
else:
    print("✅ Relatively balanced classes")

In [ ]:
# Merge datasets to create feature set
# Start with reviews and orders
ml_data = reviews[['order_id', 'review_score', 'is_high_review']].merge(
    orders, on='order_id', how='inner'
)

# Add order items information
order_items_agg = order_items.groupby('order_id').agg({
    'order_item_id': 'count',  # number of items
    'product_id': 'nunique',   # number of unique products
    'price': ['sum', 'mean', 'std'],
    'freight_value': ['sum', 'mean']
}).round(2)

order_items_agg.columns = [
    'total_items', 'unique_products', 'total_price', 'avg_item_price', 'price_std',
    'total_freight', 'avg_freight'
]
order_items_agg['price_std'] = order_items_agg['price_std'].fillna(0)  # Single item orders

ml_data = ml_data.merge(order_items_agg, on='order_id', how='left')

# Add payment information
payments_agg = payments.groupby('order_id').agg({
    'payment_sequential': 'count',  # number of payment methods
    'payment_type': lambda x: x.mode().iloc[0] if len(x) > 0 else 'unknown',  # most common payment type
    'payment_installments': 'mean',
    'payment_value': 'sum'
}).round(2)

payments_agg.columns = ['payment_methods_count', 'primary_payment_type', 'avg_installments', 'total_payment']

ml_data = ml_data.merge(payments_agg, on='order_id', how='left')

# Add customer information
ml_data = ml_data.merge(customers, on='customer_id', how='left')

print(f"ML dataset shape after merging: {ml_data.shape}")
print(f"Missing values summary:")
print(ml_data.isnull().sum().sort_values(ascending=False).head(10))

In [ ]:
# Feature Engineering - Create meaningful features from the data

# Convert datetime columns
datetime_cols = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date', 
                'order_estimated_delivery_date']

for col in datetime_cols:
    ml_data[col] = pd.to_datetime(ml_data[col])

# Delivery performance features
ml_data['delivery_days'] = (ml_data['order_delivered_customer_date'] - 
                           ml_data['order_purchase_timestamp']).dt.days
ml_data['estimated_delivery_days'] = (ml_data['order_estimated_delivery_date'] - 
                                     ml_data['order_purchase_timestamp']).dt.days
ml_data['delivery_delay_days'] = (ml_data['order_delivered_customer_date'] - 
                                 ml_data['order_estimated_delivery_date']).dt.days
ml_data['is_delivered_late'] = ml_data['delivery_delay_days'] > 0
ml_data['approval_delay_hours'] = (ml_data['order_approved_at'] - 
                                  ml_data['order_purchase_timestamp']).dt.total_seconds() / 3600

# Order timing features
ml_data['order_hour'] = ml_data['order_purchase_timestamp'].dt.hour
ml_data['order_dayofweek'] = ml_data['order_purchase_timestamp'].dt.dayofweek
ml_data['order_month'] = ml_data['order_purchase_timestamp'].dt.month
ml_data['is_weekend'] = ml_data['order_dayofweek'].isin([5, 6])

# Price and value features
ml_data['freight_to_price_ratio'] = ml_data['total_freight'] / (ml_data['total_price'] + 0.01)  # Avoid division by zero
ml_data['avg_item_value'] = ml_data['total_price'] / ml_data['total_items']
ml_data['is_high_value_order'] = ml_data['total_price'] > ml_data['total_price'].quantile(0.75)

# Order complexity features
ml_data['product_diversity_ratio'] = ml_data['unique_products'] / ml_data['total_items']
ml_data['is_single_item_order'] = ml_data['total_items'] == 1
ml_data['has_multiple_payments'] = ml_data['payment_methods_count'] > 1

print("Feature engineering completed!")
print(f"Dataset shape: {ml_data.shape}")

# Display some key statistics
print(f"\nKey feature statistics:")
print(f"Average delivery days: {ml_data['delivery_days'].mean():.1f}")
print(f"Late delivery rate: {ml_data['is_delivered_late'].mean()*100:.1f}%")
print(f"High value orders: {ml_data['is_high_value_order'].mean()*100:.1f}%")
print(f"Single item orders: {ml_data['is_single_item_order'].mean()*100:.1f}%")

In [ ]:
# Select features for modeling
# Only include orders that were delivered (to have complete delivery information)
delivered_data = ml_data[ml_data['order_status'] == 'delivered'].copy()

print(f"Filtering to delivered orders: {len(delivered_data)} records ({len(delivered_data)/len(ml_data)*100:.1f}% of total)")

# Define feature sets
numerical_features = [
    'total_items', 'unique_products', 'total_price', 'avg_item_price', 'price_std',
    'total_freight', 'avg_freight', 'avg_installments', 'total_payment',
    'delivery_days', 'estimated_delivery_days', 'delivery_delay_days',
    'approval_delay_hours', 'order_hour', 'order_dayofweek', 'order_month',
    'freight_to_price_ratio', 'avg_item_value', 'product_diversity_ratio'
]

categorical_features = [
    'order_status', 'primary_payment_type', 'customer_state'
]

boolean_features = [
    'is_delivered_late', 'is_weekend', 'is_high_value_order', 
    'is_single_item_order', 'has_multiple_payments'
]

# Combine all features
all_features = numerical_features + categorical_features + boolean_features

# Check for missing values in selected features
feature_missing = delivered_data[all_features].isnull().sum()
print(f"\nMissing values in selected features:")
print(feature_missing[feature_missing > 0])

# Remove rows with missing target or key features
delivered_data = delivered_data.dropna(subset=['is_high_review'] + numerical_features[:10])  # Keep most important features

print(f"Final dataset shape: {delivered_data.shape}")
print(f"Target distribution in final dataset:")
final_target_dist = delivered_data['is_high_review'].value_counts()
print(f"Low reviews: {final_target_dist[0]} ({final_target_dist[0]/len(delivered_data)*100:.1f}%)")
print(f"High reviews: {final_target_dist[1]} ({final_target_dist[1]/len(delivered_data)*100:.1f}%)")

## Exploratory Data Analysis for Model Features

In [ ]:
# Analyze relationship between key features and target
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Feature Analysis: High vs Low Reviews', fontsize=16)

# 1. Delivery delay impact
delivered_data.boxplot(column='delivery_delay_days', by='is_high_review', ax=axes[0,0])
axes[0,0].set_title('Delivery Delay vs Review Score')
axes[0,0].set_xlabel('Review Score (0=Low, 1=High)')

# 2. Order value impact
delivered_data.boxplot(column='total_price', by='is_high_review', ax=axes[0,1])
axes[0,1].set_title('Order Value vs Review Score')
axes[0,1].set_xlabel('Review Score (0=Low, 1=High)')

# 3. Freight ratio impact
delivered_data.boxplot(column='freight_to_price_ratio', by='is_high_review', ax=axes[0,2])
axes[0,2].set_title('Freight/Price Ratio vs Review Score')
axes[0,2].set_xlabel('Review Score (0=Low, 1=High)')

# 4. Delivery days impact
delivered_data.boxplot(column='delivery_days', by='is_high_review', ax=axes[1,0])
axes[1,0].set_title('Delivery Days vs Review Score')
axes[1,0].set_xlabel('Review Score (0=Low, 1=High)')

# 5. Number of items impact
delivered_data.boxplot(column='total_items', by='is_high_review', ax=axes[1,1])
axes[1,1].set_title('Number of Items vs Review Score')
axes[1,1].set_xlabel('Review Score (0=Low, 1=High)')

# 6. Installments impact
delivered_data.boxplot(column='avg_installments', by='is_high_review', ax=axes[1,2])
axes[1,2].set_title('Average Installments vs Review Score')
axes[1,2].set_xlabel('Review Score (0=Low, 1=High)')

plt.tight_layout()
plt.show()

# Statistical analysis of key differences
print("\nMean differences between high and low review orders:")
comparison_features = ['delivery_delay_days', 'total_price', 'freight_to_price_ratio', 
                      'delivery_days', 'total_items', 'avg_installments']

for feature in comparison_features:
    low_mean = delivered_data[delivered_data['is_high_review']==0][feature].mean()
    high_mean = delivered_data[delivered_data['is_high_review']==1][feature].mean()
    diff_pct = ((high_mean - low_mean) / low_mean * 100) if low_mean != 0 else 0
    print(f"{feature}: Low={low_mean:.2f}, High={high_mean:.2f}, Diff={diff_pct:+.1f}%")

In [ ]:
# Analyze categorical features
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Payment type vs review score
payment_review = pd.crosstab(delivered_data['primary_payment_type'], 
                           delivered_data['is_high_review'], normalize='index')
payment_review.plot(kind='bar', ax=axes[0], title='Payment Type vs Review Score')
axes[0].set_xlabel('Payment Type')
axes[0].set_ylabel('Proportion')
axes[0].legend(['Low Review', 'High Review'])

# Late delivery vs review score
late_review = pd.crosstab(delivered_data['is_delivered_late'], 
                         delivered_data['is_high_review'], normalize='index')
late_review.plot(kind='bar', ax=axes[1], title='Late Delivery vs Review Score')
axes[1].set_xlabel('Delivered Late')
axes[1].set_ylabel('Proportion')
axes[1].legend(['Low Review', 'High Review'])

# Weekend orders vs review score
weekend_review = pd.crosstab(delivered_data['is_weekend'], 
                           delivered_data['is_high_review'], normalize='index')
weekend_review.plot(kind='bar', ax=axes[2], title='Weekend Order vs Review Score')
axes[2].set_xlabel('Weekend Order')
axes[2].set_ylabel('Proportion')
axes[2].legend(['Low Review', 'High Review'])

plt.tight_layout()
plt.show()

# Print key insights
late_delivery_impact = delivered_data.groupby('is_delivered_late')['is_high_review'].mean()
print(f"\nKey Insights:")
print(f"On-time delivery high review rate: {late_delivery_impact[False]*100:.1f}%")
print(f"Late delivery high review rate: {late_delivery_impact[True]*100:.1f}%")
print(f"Late delivery impact: {(late_delivery_impact[False] - late_delivery_impact[True])*100:.1f} percentage point difference")

## Model Training and Evaluation

In [ ]:
# Prepare data for modeling
# Select final feature set (removing features with too many missing values)
final_features = [
    # Numerical features
    'total_items', 'unique_products', 'total_price', 'avg_item_price',
    'total_freight', 'avg_freight', 'avg_installments',
    'delivery_days', 'delivery_delay_days', 'order_hour', 'order_month',
    'freight_to_price_ratio', 'avg_item_value', 'product_diversity_ratio',
    # Boolean features (will be treated as numerical)
    'is_delivered_late', 'is_weekend', 'is_high_value_order', 
    'is_single_item_order', 'has_multiple_payments'
]

# Add important categorical features (encoded)
categorical_for_model = ['primary_payment_type', 'customer_state']

# Prepare feature matrix
X_numerical = delivered_data[final_features].fillna(0)

# Encode categorical features
X_categorical = pd.get_dummies(delivered_data[categorical_for_model], prefix=categorical_for_model)

# Combine features
X = pd.concat([X_numerical, X_categorical], axis=1)
y = delivered_data['is_high_review']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Feature names: {X.columns.tolist()[:10]}...")  # Show first 10 features

# Check for any remaining missing values
missing_check = X.isnull().sum().sum()
print(f"Missing values in feature matrix: {missing_check}")

if missing_check > 0:
    X = X.fillna(0)  # Fill any remaining missing values with 0
    print("Filled remaining missing values with 0")

In [ ]:
# Train-Test Split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Check class distribution in splits
print(f"\nTraining set class distribution:")
train_dist = y_train.value_counts()
print(f"Low: {train_dist[0]} ({train_dist[0]/len(y_train)*100:.1f}%)")
print(f"High: {train_dist[1]} ({train_dist[1]/len(y_train)*100:.1f}%)")

print(f"\nTest set class distribution:")
test_dist = y_test.value_counts()
print(f"Low: {test_dist[0]} ({test_dist[0]/len(y_test)*100:.1f}%)")
print(f"High: {test_dist[1]} ({test_dist[1]/len(y_test)*100:.1f}%)")

In [ ]:
# Train multiple models and compare performance
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
}

# Scale features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_results = {}

print("Training models...\n")

for name, model in models.items():
    print(f"Training {name}...")
    
    # Use scaled data for Logistic Regression, original for tree-based models
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    model_results[name] = {
        'model': model,
        'predictions': y_pred,
        'probabilities': y_pred_proba,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC-ROC: {auc:.4f}")
    print()

# Find best model based on F1-score (good balance for slightly imbalanced data)
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['f1'])
print(f"Best performing model: {best_model_name} (F1-Score: {model_results[best_model_name]['f1']:.4f})")

In [ ]:
# Detailed evaluation of the best model
best_model = model_results[best_model_name]
best_predictions = best_model['predictions']
best_probabilities = best_model['probabilities']

# Confusion Matrix
cm = confusion_matrix(y_test, best_predictions)
plt.figure(figsize=(12, 5))

# Plot 1: Confusion Matrix
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low Review', 'High Review'],
            yticklabels=['Low Review', 'High Review'])
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Plot 2: ROC Curve
plt.subplot(1, 2, 2)
fpr, tpr, _ = roc_curve(y_test, best_probabilities)
plt.plot(fpr, tpr, linewidth=2, label=f'{best_model_name} (AUC = {best_model["auc"]:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed classification report
print(f"\nDetailed Classification Report for {best_model_name}:")
print("=" * 50)
print(classification_report(y_test, best_predictions, 
                          target_names=['Low Review (1-3)', 'High Review (4-5)']))

# Calculate additional insights
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
npv = tn / (tn + fn)  # Negative Predictive Value

print(f"\nAdditional Metrics:")
print(f"Specificity (True Negative Rate): {specificity:.4f}")
print(f"Negative Predictive Value: {npv:.4f}")
print(f"False Positive Rate: {fp/(fp+tn):.4f}")
print(f"False Negative Rate: {fn/(fn+tp):.4f}")

## Feature Importance Analysis

In [ ]:
# Feature importance analysis (works best with tree-based models)
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    # Get feature importances
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    
    plt.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 20 Feature Importances - {best_model_name}')
    plt.gca().invert_yaxis()
    
    # Add value labels
    for i, v in enumerate(top_features['importance']):
        plt.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 10 Most Important Features for {best_model_name}:")
    print("=" * 60)
    for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")
        
else:
    # For Logistic Regression, show coefficient magnitudes
    feature_coefs = pd.DataFrame({
        'feature': X.columns,
        'coefficient': abs(best_model['model'].coef_[0])
    }).sort_values('coefficient', ascending=False)
    
    plt.figure(figsize=(12, 8))
    top_features = feature_coefs.head(20)
    
    plt.barh(range(len(top_features)), top_features['coefficient'], color='lightcoral')
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Absolute Coefficient Value')
    plt.title(f'Top 20 Feature Coefficients (Absolute) - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 10 Most Important Features for {best_model_name}:")
    print("=" * 60)
    for i, (_, row) in enumerate(feature_coefs.head(10).iterrows(), 1):
        original_coef = best_model['model'].coef_[0][X.columns.get_loc(row['feature'])]
        direction = "increases" if original_coef > 0 else "decreases"
        print(f"{i:2d}. {row['feature']:<25} {row['coefficient']:.4f} ({direction} probability)")

In [ ]:
# Business interpretation of key features
print("\n" + "="*80)
print("BUSINESS INTERPRETATION OF KEY FEATURES")
print("="*80)

# Get top features based on model type
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    top_features_list = feature_importance.head(5)['feature'].tolist()
    importance_values = feature_importance.head(5)['importance'].tolist()
else:
    top_features_list = feature_coefs.head(5)['feature'].tolist()
    importance_values = feature_coefs.head(5)['coefficient'].tolist()

feature_interpretations = {
    'is_delivered_late': 'Late delivery is the strongest predictor of low reviews',
    'delivery_delay_days': 'Number of days late significantly impacts customer satisfaction',
    'delivery_days': 'Longer overall delivery time correlates with lower satisfaction',
    'freight_to_price_ratio': 'High shipping costs relative to product price hurt satisfaction',
    'total_price': 'Order value affects review patterns (high-value customers may be more critical)',
    'avg_installments': 'Payment terms influence customer satisfaction',
    'total_items': 'Order complexity (number of items) impacts delivery success',
    'order_hour': 'Time of day when order was placed affects logistics performance',
    'avg_item_value': 'Individual item value influences customer expectations',
    'is_weekend': 'Weekend orders may have different processing patterns'
}

print(f"\nTop predictive factors for customer satisfaction:")
for i, (feature, importance) in enumerate(zip(top_features_list, importance_values), 1):
    interpretation = feature_interpretations.get(feature, 'Custom feature requiring domain analysis')
    print(f"{i}. {feature} (importance: {importance:.4f})")
    print(f"   → {interpretation}")
    print()

# Calculate business impact metrics
late_delivery_impact = delivered_data.groupby('is_delivered_late')['is_high_review'].mean()
high_freight_impact = delivered_data.groupby(delivered_data['freight_to_price_ratio'] > 0.2)['is_high_review'].mean()

print(f"Key Business Metrics:")
print(f"• On-time delivery satisfaction: {late_delivery_impact[False]*100:.1f}%")
print(f"• Late delivery satisfaction: {late_delivery_impact[True]*100:.1f}%")
print(f"• Impact of late delivery: {(late_delivery_impact[False] - late_delivery_impact[True])*100:.1f} percentage points")
print(f"• High freight ratio (>20%) satisfaction: {high_freight_impact[True]*100:.1f}%")
print(f"• Low freight ratio (≤20%) satisfaction: {high_freight_impact[False]*100:.1f}%")

## Model Performance Summary and Business Applications

In [ ]:
# Cross-validation to assess model stability
print("Performing cross-validation analysis...\n")

if best_model_name == 'Logistic Regression':
    cv_scores = cross_val_score(best_model['model'], X_train_scaled, y_train, cv=5, scoring='f1')
else:
    cv_scores = cross_val_score(best_model['model'], X_train, y_train, cv=5, scoring='f1')

print(f"Cross-Validation Results ({best_model_name}):")
print(f"F1-Score: {cv_scores.mean():.4f} ± {cv_scores.std()*2:.4f}")
print(f"Individual fold scores: {[f'{score:.4f}' for score in cv_scores]}")

# Model stability check
if cv_scores.std() < 0.02:
    print("✅ Model shows good stability across folds")
elif cv_scores.std() < 0.05:
    print("⚠️  Moderate variability across folds")
else:
    print("❌ High variability - model may be unstable")

# Business value calculation
print(f"\n" + "="*60)
print("BUSINESS VALUE ANALYSIS")
print("="*60)

# Calculate potential impact
total_orders = len(y_test)
predicted_low_reviews = (best_predictions == 0).sum()
actual_low_reviews = (y_test == 0).sum()
true_positives = tp  # Correctly identified high reviews
false_negatives = fn  # Missed low reviews (high risk)
true_negatives = tn  # Correctly identified low reviews
false_positives = fp  # Incorrectly flagged as low review

print(f"Model Performance on Test Set ({total_orders:,} orders):")
print(f"• Correctly identified high reviews: {true_positives:,} orders")
print(f"• Correctly identified low reviews: {true_negatives:,} orders")
print(f"• Missed low reviews (high risk): {false_negatives:,} orders")
print(f"• False alarms: {false_positives:,} orders")

# Business impact estimates
intervention_cost_per_order = 10  # Assumed cost to intervene (proactive customer service)
churn_cost_per_customer = 150    # Assumed customer lifetime value loss
intervention_success_rate = 0.3  # Assumed success rate of intervention

# Calculate costs and savings
intervention_cost = true_negatives * intervention_cost_per_order
false_alarm_cost = false_positives * intervention_cost_per_order
missed_opportunity_cost = false_negatives * churn_cost_per_customer * intervention_success_rate
prevented_churn_value = true_negatives * churn_cost_per_customer * intervention_success_rate

net_value = prevented_churn_value - intervention_cost - false_alarm_cost

print(f"\nEstimated Business Impact (Test Set):")
print(f"• Intervention cost: R$ {intervention_cost:,.2f}")
print(f"• False alarm cost: R$ {false_alarm_cost:,.2f}")
print(f"• Prevented churn value: R$ {prevented_churn_value:,.2f}")
print(f"• Missed opportunity cost: R$ {missed_opportunity_cost:,.2f}")
print(f"• Net business value: R$ {net_value:,.2f}")

roi = (net_value / (intervention_cost + false_alarm_cost)) * 100
print(f"• ROI: {roi:.1f}%")

if net_value > 0:
    print("✅ Model provides positive business value")
else:
    print("❌ Model needs improvement or different intervention strategy")

## Model Limitations and Recommendations

In [ ]:
print("="*80)
print("MODEL LIMITATIONS AND IMPROVEMENT OPPORTUNITIES")
print("="*80)

# Class imbalance analysis
class_distribution = y.value_counts(normalize=True)
print(f"\n1. CLASS IMBALANCE ANALYSIS:")
print(f"   • High reviews: {class_distribution[1]*100:.1f}%")
print(f"   • Low reviews: {class_distribution[0]*100:.1f}%")
print(f"   • Imbalance ratio: {class_distribution[1]/class_distribution[0]:.2f}:1")

if class_distribution[1]/class_distribution[0] > 3:
    print("   ⚠️  Significant class imbalance - model may be biased toward predicting high reviews")
    print("   💡 Recommendation: Use SMOTE, cost-sensitive learning, or threshold tuning")

# Feature coverage analysis
missing_data_pct = delivered_data[final_features].isnull().sum().sum() / (len(delivered_data) * len(final_features)) * 100
print(f"\n2. DATA QUALITY:")
print(f"   • Missing data: {missing_data_pct:.2f}% of feature values")
print(f"   • Dataset coverage: {len(delivered_data):,} orders ({len(delivered_data)/len(ml_data)*100:.1f}% of total)")

# Temporal bias
date_range = delivered_data['order_purchase_timestamp'].dt.date
print(f"\n3. TEMPORAL LIMITATIONS:")
print(f"   • Date range: {date_range.min()} to {date_range.max()}")
print(f"   • Time span: {(date_range.max() - date_range.min()).days} days")
print(f"   ⚠️  Model trained on historical data - may not capture current market conditions")
print(f"   💡 Recommendation: Implement model retraining pipeline with recent data")

# Feature engineering opportunities
print(f"\n4. MISSING FEATURES (Potential Improvements):")
missing_features = [
    "Product category satisfaction history",
    "Seller performance metrics",
    "Customer purchase history/loyalty",
    "Seasonal/holiday effects",
    "Geographic distance (customer-seller)",
    "Product reviews/ratings",
    "Competitor pricing data",
    "Customer demographics",
    "Weather/external factors",
    "Marketing campaign exposure"
]

for i, feature in enumerate(missing_features, 1):
    print(f"   {i:2d}. {feature}")

print(f"\n5. SCALABILITY CONSIDERATIONS:")
print(f"   • Current feature count: {X.shape[1]}")
print(f"   • Training time: Acceptable for current dataset size")
print(f"   • Memory usage: {X.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"   💡 For production: Consider feature selection, model simplification, or ensemble approaches")

print(f"\n6. BUSINESS DEPLOYMENT RECOMMENDATIONS:")
recommendations = [
    "Implement real-time scoring for new orders",
    "Set up A/B testing framework for intervention strategies",
    "Create monitoring dashboard for model performance drift",
    "Establish feedback loop to capture intervention outcomes",
    "Develop tiered intervention strategy based on prediction confidence",
    "Integration with customer service workflows",
    "Regular model retraining (monthly/quarterly)",
    "Expand to predict specific review scores (1-5) instead of binary"
]

for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print(f"\n" + "="*80)
print(f"FINAL MODEL SUMMARY")
print(f"="*80)
print(f"Best Model: {best_model_name}")
print(f"Accuracy: {best_model['accuracy']:.3f} | Precision: {best_model['precision']:.3f} | Recall: {best_model['recall']:.3f}")
print(f"F1-Score: {best_model['f1']:.3f} | AUC-ROC: {best_model['auc']:.3f}")
print(f"Cross-validation F1: {cv_scores.mean():.3f} ± {cv_scores.std()*2:.3f}")
print(f"Estimated ROI: {roi:.1f}%")
print(f"Deployment Ready: {'✅ Yes' if best_model['f1'] > 0.7 and cv_scores.std() < 0.05 else '⚠️  Needs improvement'}")